# Step 7: Attention Mechanism - LLM의 핵심 원리

Attention은 "Attention is All You Need" 논문에서 소개된 혁신적인 메커니즘으로, 현대 LLM의 핵심입니다.

## 학습 목표
1. Attention의 직관적 이해
2. Query, Key, Value 개념 마스터
3. Scaled Dot-Product Attention 구현
4. Multi-Head Attention 이해
5. Positional Encoding의 필요성

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
import math

# 시각화 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

# 재현성을 위한 시드 설정
torch.manual_seed(42)
np.random.seed(42)

## 1. Attention의 직관적 이해

Attention은 "어디에 집중할 것인가"를 학습하는 메커니즘입니다.

### 예시: 문장 번역
"나는 사과를 좋아한다" → "I love apples"

- "나는" → "I"에 집중
- "좋아한다" → "love"에 집중
- "사과를" → "apples"에 집중

In [ ]:
# 간단한 Attention 예제
def simple_attention_example():
    # 예시 문장
    source = ["나는", "사과를", "좋아한다"]
    target = ["I", "love", "apples"]
    
    # 수동으로 만든 attention 점수 (실제로는 학습됨)
    attention_scores = np.array([
        [0.9, 0.05, 0.05],  # I → 나는(0.9), 사과를(0.05), 좋아한다(0.05)
        [0.1, 0.1, 0.8],    # love → 나는(0.1), 사과를(0.1), 좋아한다(0.8)
        [0.05, 0.9, 0.05]   # apples → 나는(0.05), 사과를(0.9), 좋아한다(0.05)
    ])
    
    # 시각화
    plt.figure(figsize=(8, 6))
    sns.heatmap(attention_scores, 
                xticklabels=source, 
                yticklabels=target, 
                cmap='Blues', 
                annot=True, 
                fmt='.2f',
                cbar_kws={'label': 'Attention Weight'})
    plt.title('Attention Weights: 번역 예시')
    plt.xlabel('Source (한국어)')
    plt.ylabel('Target (영어)')
    plt.tight_layout()
    plt.show()

simple_attention_example()

## 2. Query, Key, Value의 개념

Attention의 핵심은 Query(Q), Key(K), Value(V)입니다.

### 직관적 비유: 도서관에서 책 찾기
- **Query**: "Python 프로그래밍에 관한 책을 찾고 싶어" (내가 찾는 것)
- **Key**: 각 책의 제목, 카테고리 (검색 가능한 정보)
- **Value**: 실제 책의 내용 (가져올 정보)

Query와 Key의 유사도가 높을수록, 해당 Value에 더 많이 집중합니다.

In [ ]:
# Query, Key, Value 예제
def qkv_example():
    # 간단한 예제: 3개의 단어, 4차원 임베딩
    seq_len = 3
    d_model = 4
    
    # 입력 시퀀스 (예: "나는 학생이다"의 임베딩)
    x = torch.randn(seq_len, d_model)
    
    # Q, K, V 변환 행렬
    W_q = torch.randn(d_model, d_model)
    W_k = torch.randn(d_model, d_model)
    W_v = torch.randn(d_model, d_model)
    
    # Q, K, V 계산
    Q = x @ W_q  # Query
    K = x @ W_k  # Key
    V = x @ W_v  # Value
    
    print("입력 형태:", x.shape)
    print("Q 형태:", Q.shape)
    print("K 형태:", K.shape)
    print("V 형태:", V.shape)
    
    # 시각화
    fig, axes = plt.subplots(1, 4, figsize=(15, 3))
    
    matrices = [x, Q, K, V]
    titles = ['Input (X)', 'Query (Q)', 'Key (K)', 'Value (V)']
    
    for i, (matrix, title) in enumerate(zip(matrices, titles)):
        im = axes[i].imshow(matrix.detach().numpy(), cmap='coolwarm', aspect='auto')
        axes[i].set_title(title)
        axes[i].set_xlabel('Dimension')
        axes[i].set_ylabel('Position')
        plt.colorbar(im, ax=axes[i])
    
    plt.tight_layout()
    plt.show()
    
    return Q, K, V

Q, K, V = qkv_example()

## 3. Scaled Dot-Product Attention

Attention의 핵심 공식:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

여기서:
- $QK^T$: Query와 Key의 유사도 계산
- $\sqrt{d_k}$: 스케일링 (값이 너무 커지는 것을 방지)
- softmax: 확률 분포로 변환
- V: Value에 가중치 적용

In [ ]:
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Scaled Dot-Product Attention 구현
    
    Args:
        Q: Query (seq_len, d_k)
        K: Key (seq_len, d_k)
        V: Value (seq_len, d_v)
        mask: 마스킹 (선택사항)
    
    Returns:
        attention_output: 어텐션 결과
        attention_weights: 어텐션 가중치
    """
    d_k = Q.size(-1)
    
    # 1. Q와 K의 유사도 계산
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
    
    # 2. 마스킹 (선택사항)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    
    # 3. Softmax로 확률 분포 생성
    attention_weights = F.softmax(scores, dim=-1)
    
    # 4. Value에 가중치 적용
    attention_output = torch.matmul(attention_weights, V)
    
    return attention_output, attention_weights

# 예제 실행
output, weights = scaled_dot_product_attention(Q, K, V)

print("Attention 출력 형태:", output.shape)
print("Attention 가중치 형태:", weights.shape)

# Attention 가중치 시각화
plt.figure(figsize=(6, 5))
sns.heatmap(weights.detach().numpy(), 
            annot=True, 
            fmt='.3f', 
            cmap='Blues',
            xticklabels=['Pos 0', 'Pos 1', 'Pos 2'],
            yticklabels=['Pos 0', 'Pos 1', 'Pos 2'])
plt.title('Attention Weights Matrix')
plt.xlabel('Key Positions')
plt.ylabel('Query Positions')
plt.tight_layout()
plt.show()

### 3.1 단계별 Attention 계산 시각화

In [ ]:
def visualize_attention_steps():
    # 더 간단한 예제로 단계별 설명
    seq_len = 4
    d_k = 3
    
    # 예시 데이터
    Q = torch.tensor([[1.0, 0.0, 1.0],
                      [0.0, 1.0, 0.0],
                      [1.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0]])
    
    K = torch.tensor([[1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0],
                      [1.0, 1.0, 0.0]])
    
    V = torch.tensor([[1.0, 0.0],
                      [0.0, 1.0],
                      [1.0, 1.0],
                      [0.5, 0.5]])
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    # Step 1: Q와 K 표시
    axes[0, 0].imshow(Q, cmap='Blues', aspect='auto')
    axes[0, 0].set_title('Query (Q)')
    axes[0, 0].set_ylabel('Positions')
    axes[0, 0].set_xlabel('Dimensions')
    
    axes[0, 1].imshow(K, cmap='Oranges', aspect='auto')
    axes[0, 1].set_title('Key (K)')
    axes[0, 1].set_ylabel('Positions')
    axes[0, 1].set_xlabel('Dimensions')
    
    # Step 2: QK^T 계산
    scores = torch.matmul(Q, K.transpose(0, 1))
    axes[0, 2].imshow(scores, cmap='Greens', aspect='auto')
    axes[0, 2].set_title(r'$QK^T$ (Similarity Scores)')
    axes[0, 2].set_ylabel('Query Positions')
    axes[0, 2].set_xlabel('Key Positions')
    
    # Step 3: Scaling
    scaled_scores = scores / math.sqrt(d_k)
    axes[1, 0].imshow(scaled_scores, cmap='Greens', aspect='auto')
    axes[1, 0].set_title(r'$\frac{QK^T}{\sqrt{d_k}}$ (Scaled Scores)')
    axes[1, 0].set_ylabel('Query Positions')
    axes[1, 0].set_xlabel('Key Positions')
    
    # Step 4: Softmax
    attention_weights = F.softmax(scaled_scores, dim=-1)
    im = axes[1, 1].imshow(attention_weights, cmap='Reds', aspect='auto')
    axes[1, 1].set_title('Softmax(Scaled Scores)')
    axes[1, 1].set_ylabel('Query Positions')
    axes[1, 1].set_xlabel('Key Positions')
    
    # 값 표시
    for i in range(seq_len):
        for j in range(seq_len):
            axes[1, 1].text(j, i, f'{attention_weights[i, j]:.2f}', 
                           ha='center', va='center')
    
    # Step 5: Attention Output
    output = torch.matmul(attention_weights, V)
    axes[1, 2].imshow(output, cmap='Purples', aspect='auto')
    axes[1, 2].set_title('Attention Output')
    axes[1, 2].set_ylabel('Positions')
    axes[1, 2].set_xlabel('Output Dimensions')
    
    plt.tight_layout()
    plt.show()

visualize_attention_steps()

## 4. Self-Attention

Self-Attention은 같은 시퀀스 내에서 각 위치가 다른 모든 위치를 참조하는 메커니즘입니다.

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, d_model):
        super(SelfAttention, self).__init__()
        self.d_model = d_model
        
        # Q, K, V를 위한 선형 변환
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        
    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # Scaled dot-product attention
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        attention_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attention_weights, V)
        
        return output, attention_weights

# Self-Attention 예제
def self_attention_example():
    # 예시 문장: "나는 학교에 간다"
    words = ["나는", "학교에", "간다"]
    seq_len = len(words)
    d_model = 8
    
    # 임의의 단어 임베딩
    x = torch.randn(1, seq_len, d_model)
    
    # Self-Attention 적용
    self_attn = SelfAttention(d_model)
    output, weights = self_attn(x)
    
    # 가중치 시각화
    plt.figure(figsize=(6, 5))
    sns.heatmap(weights[0].detach().numpy(), 
                annot=True, 
                fmt='.3f',
                xticklabels=words,
                yticklabels=words,
                cmap='YlOrRd')
    plt.title('Self-Attention Weights')
    plt.xlabel('참조하는 단어')
    plt.ylabel('현재 단어')
    plt.tight_layout()
    plt.show()
    
    # 해석
    print("\nSelf-Attention 해석:")
    for i, word in enumerate(words):
        print(f"\n'{word}'가 주목하는 단어:")
        for j, ref_word in enumerate(words):
            print(f"  - '{ref_word}': {weights[0, i, j]:.3f}")

self_attention_example()

## 5. Multi-Head Attention

Multi-Head Attention은 여러 개의 attention을 병렬로 수행하여 다양한 관점에서 정보를 추출합니다.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # 각 헤드를 위한 선형 변환
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def forward(self, x, mask=None):
        batch_size, seq_len, _ = x.size()
        
        # 1. Q, K, V 계산
        Q = self.W_q(x)  # (batch_size, seq_len, d_model)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # 2. 헤드별로 분리
        Q = Q.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        # 현재 형태: (batch_size, num_heads, seq_len, d_k)
        
        # 3. Scaled Dot-Product Attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention_weights = F.softmax(scores, dim=-1)
        context = torch.matmul(attention_weights, V)
        
        # 4. 헤드 합치기
        context = context.transpose(1, 2).contiguous().view(
            batch_size, seq_len, self.d_model
        )
        
        # 5. 최종 선형 변환
        output = self.W_o(context)
        
        return output, attention_weights

# Multi-Head Attention 시각화
def visualize_multihead_attention():
    seq_len = 4
    d_model = 8
    num_heads = 2
    
    # 입력 생성
    x = torch.randn(1, seq_len, d_model)
    
    # Multi-Head Attention
    mha = MultiHeadAttention(d_model, num_heads)
    output, weights = mha(x)
    
    # 각 헤드의 attention 가중치 시각화
    fig, axes = plt.subplots(1, num_heads, figsize=(12, 5))
    
    for head in range(num_heads):
        ax = axes[head]
        sns.heatmap(weights[0, head].detach().numpy(),
                   annot=True,
                   fmt='.2f',
                   cmap='Blues',
                   ax=ax)
        ax.set_title(f'Head {head + 1}')
        ax.set_xlabel('Key Positions')
        ax.set_ylabel('Query Positions')
    
    plt.suptitle('Multi-Head Attention Weights', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print(f"입력 형태: {x.shape}")
    print(f"출력 형태: {output.shape}")
    print(f"\n각 헤드는 서로 다른 패턴에 주목합니다!")

visualize_multihead_attention()

## 6. Positional Encoding

Attention은 순서 정보가 없으므로, 위치 정보를 추가해야 합니다.

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # 위치 인코딩 계산
        pe = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len).unsqueeze(1).float()
        
        # 주파수 계산
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           -(math.log(10000.0) / d_model))
        
        # 사인과 코사인 적용
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len]

# Positional Encoding 시각화
def visualize_positional_encoding():
    d_model = 128
    max_seq_len = 100
    
    pe = PositionalEncoding(d_model, max_seq_len)
    
    # 위치 인코딩 값 가져오기
    encoding = pe.pe[0, :50, :].numpy()
    
    # 시각화
    plt.figure(figsize=(12, 6))
    plt.imshow(encoding.T, cmap='RdBu', aspect='auto')
    plt.colorbar(label='Value')
    plt.xlabel('Position')
    plt.ylabel('Dimension')
    plt.title('Positional Encoding Pattern')
    plt.tight_layout()
    plt.show()
    
    # 특정 차원의 패턴 보기
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.ravel()
    
    dimensions = [0, 1, 10, 11]
    for i, dim in enumerate(dimensions):
        axes[i].plot(encoding[:, dim])
        axes[i].set_title(f'Dimension {dim} ({"sin" if dim % 2 == 0 else "cos"})')
        axes[i].set_xlabel('Position')
        axes[i].set_ylabel('Value')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_positional_encoding()

## 7. Masked Attention (Causal Attention)

언어 모델에서는 미래 정보를 보지 못하도록 마스킹을 적용합니다.

In [ ]:
def create_causal_mask(seq_len):
    """
    Causal mask 생성 (미래 정보 차단)
    """
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
    return mask == 0

def masked_attention_example():
    seq_len = 5
    d_model = 8
    
    # 예시 문장: "나는 오늘 학교에 갔다"
    words = ["나는", "오늘", "학교에", "갔다", "<END>"]
    
    # 입력과 마스크 생성
    x = torch.randn(1, seq_len, d_model)
    mask = create_causal_mask(seq_len)
    
    # Attention 계산
    self_attn = SelfAttention(d_model)
    Q = self_attn.W_q(x)
    K = self_attn.W_k(x)
    V = self_attn.W_v(x)
    
    # Masked attention
    scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_model)
    scores = scores.masked_fill(mask == 0, -1e9)
    attention_weights = F.softmax(scores, dim=-1)
    
    # 시각화
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 1. Causal Mask
    axes[0].imshow(mask.float(), cmap='gray', aspect='auto')
    axes[0].set_title('Causal Mask\n(흰색=볼 수 있음, 검은색=볼 수 없음)')
    axes[0].set_xlabel('Key Position')
    axes[0].set_ylabel('Query Position')
    
    # 라벨 추가
    for i in range(seq_len):
        axes[0].set_xticks(range(seq_len))
        axes[0].set_xticklabels(words, rotation=45)
        axes[0].set_yticks(range(seq_len))
        axes[0].set_yticklabels(words)
    
    # 2. Raw Scores
    raw_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_model)
    im = axes[1].imshow(raw_scores[0].detach().numpy(), cmap='Blues', aspect='auto')
    axes[1].set_title('Raw Attention Scores')
    axes[1].set_xticks(range(seq_len))
    axes[1].set_xticklabels(words, rotation=45)
    axes[1].set_yticks(range(seq_len))
    axes[1].set_yticklabels(words)
    plt.colorbar(im, ax=axes[1])
    
    # 3. Masked Attention Weights
    im = axes[2].imshow(attention_weights[0].detach().numpy(), cmap='Reds', aspect='auto')
    axes[2].set_title('Masked Attention Weights')
    axes[2].set_xticks(range(seq_len))
    axes[2].set_xticklabels(words, rotation=45)
    axes[2].set_yticks(range(seq_len))
    axes[2].set_yticklabels(words)
    plt.colorbar(im, ax=axes[2])
    
    # 값 표시
    for i in range(seq_len):
        for j in range(seq_len):
            if mask[i, j]:
                axes[2].text(j, i, f'{attention_weights[0, i, j]:.2f}', 
                           ha='center', va='center', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    print("\nCausal Attention 설명:")
    print("- '나는'은 자기 자신만 볼 수 있음")
    print("- '오늘'은 '나는'과 자기 자신을 볼 수 있음")
    print("- '학교에'는 '나는', '오늘', 자기 자신을 볼 수 있음")
    print("- 이런 식으로 미래 단어는 볼 수 없음!")

masked_attention_example()

## 8. 실제 문장에서 Attention 이해하기

In [ ]:
def real_sentence_attention():
    # 예시 문장과 간단한 임베딩
    sentence = "The cat sat on the mat"
    words = sentence.split()
    vocab = {word: i for i, word in enumerate(set(words))}
    
    # 원-핫 인코딩 후 임베딩
    d_model = 8
    embedding = nn.Embedding(len(vocab), d_model)
    
    # 단어를 인덱스로 변환
    indices = torch.tensor([vocab[word] for word in words])
    x = embedding(indices).unsqueeze(0)  # (1, seq_len, d_model)
    
    # Self-Attention 적용
    self_attn = SelfAttention(d_model)
    output, weights = self_attn(x)
    
    # Attention 패턴 시각화
    plt.figure(figsize=(8, 6))
    sns.heatmap(weights[0].detach().numpy(),
                xticklabels=words,
                yticklabels=words,
                annot=True,
                fmt='.3f',
                cmap='YlOrRd',
                cbar_kws={'label': 'Attention Weight'})
    plt.title('Self-Attention on: "The cat sat on the mat"')
    plt.xlabel('Attending to (참조하는 단어)')
    plt.ylabel('From (현재 단어)')
    plt.tight_layout()
    plt.show()
    
    # 특정 단어의 attention 분석
    word_idx = 1  # "cat"
    print(f"\n'{words[word_idx]}'이(가) 주목하는 단어들:")
    attention_scores = weights[0, word_idx].detach().numpy()
    for i, (word, score) in enumerate(zip(words, attention_scores)):
        print(f"  {word}: {score:.3f} {'***' if score > 0.2 else ''}")

real_sentence_attention()

## 9. Attention의 장점과 특징

In [ ]:
def attention_advantages():
    # Attention의 장점 시각화
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. Long-range dependencies
    ax = axes[0, 0]
    sentence = "The student who studied hard passed the exam"
    words = sentence.split()
    positions = list(range(len(words)))
    
    # RNN vs Attention 경로
    ax.scatter(positions, [1]*len(words), s=100)
    for i, word in enumerate(words):
        ax.text(i, 1, word, ha='center', va='bottom', rotation=45)
    
    # RNN 경로 (순차적)
    for i in range(len(words)-1):
        ax.arrow(i, 0.5, 0.8, 0, head_width=0.1, head_length=0.1, 
                fc='blue', alpha=0.5)
    
    # Attention 경로 (직접 연결)
    ax.arrow(1, 1.5, 5, 0, head_width=0.1, head_length=0.2, 
            fc='red', alpha=0.7, linestyle='--')
    
    ax.set_ylim(0, 2)
    ax.set_title('Long-range Dependencies\nRNN(파란색) vs Attention(빨간색)')
    ax.axis('off')
    
    # 2. Parallelization
    ax = axes[0, 1]
    ax.text(0.5, 0.8, 'RNN: Sequential Processing', ha='center', fontsize=12)
    ax.text(0.5, 0.7, 't=1 → t=2 → t=3 → t=4', ha='center', fontsize=10)
    ax.text(0.5, 0.5, 'Attention: Parallel Processing', ha='center', fontsize=12)
    ax.text(0.5, 0.4, 't=1, t=2, t=3, t=4 (동시에!)', ha='center', fontsize=10)
    ax.text(0.5, 0.2, '⚡ 훨씬 빠른 학습!', ha='center', fontsize=14, color='green')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('병렬 처리의 장점')
    
    # 3. Interpretability
    ax = axes[1, 0]
    attention_matrix = np.random.rand(5, 5)
    attention_matrix = attention_matrix / attention_matrix.sum(axis=1, keepdims=True)
    im = ax.imshow(attention_matrix, cmap='Blues')
    ax.set_title('Attention의 해석 가능성\n(어디에 주목하는지 볼 수 있음)')
    ax.set_xlabel('Input positions')
    ax.set_ylabel('Output positions')
    plt.colorbar(im, ax=ax)
    
    # 4. 계산 복잡도 비교
    ax = axes[1, 1]
    seq_lengths = np.array([10, 50, 100, 200, 500])
    rnn_complexity = seq_lengths  # O(n)
    attention_complexity = seq_lengths ** 2  # O(n²)
    
    ax.plot(seq_lengths, rnn_complexity, 'b-', label='RNN: O(n)', linewidth=2)
    ax.plot(seq_lengths, attention_complexity, 'r-', label='Attention: O(n²)', linewidth=2)
    ax.set_xlabel('Sequence Length')
    ax.set_ylabel('Computational Complexity')
    ax.set_title('계산 복잡도 비교')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\nAttention의 주요 장점:")
    print("1. 장거리 의존성: 멀리 떨어진 단어도 직접 참조 가능")
    print("2. 병렬 처리: 모든 위치를 동시에 계산 → 빠른 학습")
    print("3. 해석 가능성: Attention 가중치로 모델의 동작 이해 가능")
    print("\n단점:")
    print("- 메모리 사용량: O(n²) → 긴 시퀀스에서 문제")
    print("- 위치 정보 부재: Positional Encoding 필요")

attention_advantages()

## 10. 연습 문제

In [ ]:
# 문제 1: Cross-Attention 구현
# Encoder의 출력을 참조하는 Decoder attention
class CrossAttention(nn.Module):
    def __init__(self, d_model):
        super(CrossAttention, self).__init__()
        # 힌트: Q는 decoder에서, K와 V는 encoder에서 옵니다
        # TODO: 구현하기
        pass
    
    def forward(self, decoder_input, encoder_output):
        # TODO: 구현하기
        pass

# 문제 2: Relative Position Encoding
# 절대 위치가 아닌 상대 위치 인코딩 구현
def relative_position_encoding(seq_len, d_model):
    # 힌트: 각 위치 쌍 (i, j)에 대해 i-j를 인코딩
    # TODO: 구현하기
    pass

# 문제 3: Sparse Attention
# 모든 위치가 아닌 일부만 참조하는 attention
def sparse_attention_pattern(seq_len, window_size):
    # 힌트: 각 위치는 window_size 범위 내의 위치만 참조
    # TODO: 구현하기
    pass

## 정리

이번 튜토리얼에서 배운 내용:
1. **Attention의 직관적 이해**: 어디에 주목할지 학습하는 메커니즘
2. **Query, Key, Value**: 검색 시스템과 유사한 개념
3. **Scaled Dot-Product Attention**: 핵심 계산 과정
4. **Multi-Head Attention**: 다양한 관점에서 정보 추출
5. **Positional Encoding**: 순서 정보 추가
6. **Masked Attention**: 언어 모델을 위한 미래 정보 차단

### 핵심 공식 다시 보기:
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### Attention이 혁신적인 이유:
- **병렬 처리**: RNN과 달리 모든 위치를 동시에 계산
- **장거리 의존성**: 멀리 떨어진 정보도 직접 참조
- **해석 가능성**: Attention 가중치로 모델 동작 이해

다음 단계에서는 이 Attention을 사용하여 완전한 Transformer를 구축해보겠습니다!